In [2]:
# Загружаем библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from lightgbm import LGBMClassifier

In [ ]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [3]:
# Объявляем датасеты
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

In [4]:
train_df.head() # Выводим первые 5 строк

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other}


In [5]:
test_df.head() # Выводим первые 5 строк

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,None,...,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α}
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α}
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,None
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}


## Train data

In [6]:
# Удаляем столбцы
train_df = train_df.drop(columns = ["channel_code", "city", 'index_city_code', 'ogrn_days_end_month','ogrn_days_end_quarter', 'ogrn_month', 'ft_registration_date', 'min_founderpres'])

In [7]:
df_float = train_df.select_dtypes(exclude='object').reset_index()  # Создаем датафрейм со столбцами численного типа И проверяем количесво пропусков
df_float.head()                                                     # Для последующего соединения таблиц установим столбец индексов

,index,id,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,max_founderpres,ogrn_exist_months,sum_of_paym_2m,sum_of_paym_6m,...,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m
0,0,0,0.744845,0.705492,1.287207,0.748101,2.736461,2.437979,0.942275,0.536013,...,0.728507,0.186820,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947
1,1,0,1.049605,0.831916,2.458609,1.053805,2.756014,2.455375,0.645704,0.536378,...,0.728507,0.125043,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969
2,2,0,0.692653,0.740253,0.430042,0.695747,2.773674,2.472771,0.403604,0.613167,...,0.728507,0.033607,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958
3,3,1,-0.081586,-0.091860,-0.114040,-0.080890,NaN,0.176508,0.014051,0.043800,...,0.750485,-0.024561,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452
4,4,1,-0.094962,-0.100504,-0.119302,-0.094307,NaN,0.193904,-0.057593,0.035027,...,0.772463,-0.023981,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419


In [8]:
# Смотрим количество пропусков в датасете
df_float.isna().sum()

index                           0
id                              0
balance_amt_avg             65577
balance_amt_max             65577
balance_amt_min             65577
                            ...  
cnt_deb_h_oper_3m          165252
cnt_days_deb_h_oper_3m     165252
sum_cred_h_oper_3m              0
cnt_cred_h_oper_3m         165252
cnt_days_cred_h_oper_3m    165252
Length: 79, dtype: int64

In [9]:
# Создаем датафрейм с численными столбцами и заполняем пропуски средними значениями
df_float = df_float.fillna(df_float.mean()) 

In [10]:
# Создаем датафрейм с категориальными столбцами и заполняем пропуски модами
df_object = train_df.select_dtypes('object').reset_index() 
df_object = df_object.fillna(df_object.mode().iloc[0]).reset_index()

In [11]:
# Объединяем датафреймы с численными и категориальными данными
df_modifide = df_float.merge(df_object, on=['index']).drop('index', axis=1)
df_modifide.head()

,id,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,max_founderpres,ogrn_exist_months,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,...,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,level_0,date,city_type,ogrn_year,okved,segment,start_cluster,end_cluster
0,0,0.744845,0.705492,1.287207,0.748101,2.736461e+00,2.437979,0.942275,0.536013,0.511490,...,0.295984,0.967947,0,month_1,city_type_0,ogrn_year_1,okved_30,segment_1,"{α, γ}",{other}
1,0,1.049605,0.831916,2.458609,1.053805,2.756014e+00,2.455375,0.645704,0.536378,0.486425,...,0.298873,0.945969,1,month_2,city_type_0,ogrn_year_1,okved_30,segment_1,"{α, γ}",{other}
2,0,0.692653,0.740253,0.430042,0.695747,2.773674e+00,2.472771,0.403604,0.613167,0.480547,...,0.294829,0.956958,2,month_3,city_type_0,ogrn_year_1,okved_30,segment_1,"{α, γ}",{other}
3,1,-0.081586,-0.091860,-0.114040,-0.080890,1.211971e-17,0.176508,0.014051,0.043800,0.052041,...,0.253523,0.462452,3,month_1,city_type_0,ogrn_year_15,okved_5,segment_1,{other},{other}
4,1,-0.094962,-0.100504,-0.119302,-0.094307,1.211971e-17,0.193904,-0.057593,0.035027,0.033554,...,0.255545,0.495419,4,month_2,city_type_0,ogrn_year_15,okved_5,segment_1,{other},{other}


In [12]:
# Смотрим количество пропусков в датасете
df_modifide.isna().sum()

id                     0
balance_amt_avg        0
balance_amt_max        0
balance_amt_min        0
balance_amt_day_avg    0
                      ..
ogrn_year              0
okved                  0
segment                0
start_cluster          0
end_cluster            0
Length: 86, dtype: int64

In [13]:
# Заменим тип столбцов object на тип category
category_list = [
    "date", "city_type",
    "okved", "segment",
    "start_cluster", "ogrn_year",
]

df_modifide[category_list] = df_modifide[category_list].astype("category")

In [14]:
# Разбиваем датасет на обучающую и валидационную выборку
X = df_modifide.drop(["id", "date", "end_cluster"], axis=1)
y = df_modifide["end_cluster"]

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [15]:
X.head()

,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,max_founderpres,ogrn_exist_months,sum_of_paym_2m,sum_of_paym_6m,sum_of_paym_1y,sum_a_oper_1m,...,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,level_0,city_type,ogrn_year,okved,segment,start_cluster
0,0.744845,0.705492,1.287207,0.748101,2.736461e+00,2.437979,0.942275,0.536013,0.511490,-0.188835,...,0.774354,0.936506,0.295984,0.967947,0,city_type_0,ogrn_year_1,okved_30,segment_1,"{α, γ}"
1,1.049605,0.831916,2.458609,1.053805,2.756014e+00,2.455375,0.645704,0.536378,0.486425,-0.188835,...,0.696576,0.990157,0.298873,0.945969,1,city_type_0,ogrn_year_1,okved_30,segment_1,"{α, γ}"
2,0.692653,0.740253,0.430042,0.695747,2.773674e+00,2.472771,0.403604,0.613167,0.480547,-0.188835,...,0.663243,0.810065,0.294829,0.956958,2,city_type_0,ogrn_year_1,okved_30,segment_1,"{α, γ}"
3,-0.081586,-0.091860,-0.114040,-0.080890,1.211971e-17,0.176508,0.014051,0.043800,0.052041,-0.188835,...,0.785465,-0.184002,0.253523,0.462452,3,city_type_0,ogrn_year_15,okved_5,segment_1,{other}
4,-0.094962,-0.100504,-0.119302,-0.094307,1.211971e-17,0.193904,-0.057593,0.035027,0.033554,-0.188835,...,0.696576,-0.183854,0.255545,0.495419,4,city_type_0,ogrn_year_15,okved_5,segment_1,{other}


In [16]:
y.head()

0    {other}
1    {other}
2    {other}
3    {other}
4    {other}
Name: end_cluster, dtype: object

In [17]:
# Обучаем модель
model = LGBMClassifier(verbosity=-1, random_state=42, n_jobs=-1)
model.fit(x_train, y_train)

LGBMClassifier(n_jobs=-1, random_state=42, verbosity=-1)

In [18]:
# Функция для вычисления взвешенного ROC AUC
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average='weighted')
    return sum(weights * classes_roc_auc)

In [19]:
# Загружаем веса для кластеров и применяем функцию взвешенного ROC AUC к валидационным данным
cluster_weights = pd.read_excel('cluster_weights.xlsx').set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()
y_pred_proba = model.predict_proba(x_val)
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.7965829149477437

## Test data

In [20]:
# Обработка тестовой выборки: выбрать все строки, где month_4 и month_5, заполнить пропуски по столбцам, обучить модель для получения целевой переменной,
# отсортировать полученную выборку по end_cluster, заполнить start_cluster по групповой моде.

test_temp_df = test_df.query("date == 'month_4' or date == 'month_5'")
test_temp_df.head()

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,None,...,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α}
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,None,...,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α}
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,None,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
6,200002,month_4,-0.096506,0.185905,-0.125995,-0.095856,channel_code_12,city_14,city_type_0,index_city_code_78,...,0.510730,0.950774,0.545693,0.715525,0.554913,0.718798,0.445811,0.254968,0.495419,{other}


In [21]:
test_temp_df.isna().sum()

id                             0
date                           0
balance_amt_avg            24552
balance_amt_max            24552
balance_amt_min            24552
                           ...  
cnt_days_deb_h_oper_3m     56648
sum_cred_h_oper_3m             0
cnt_cred_h_oper_3m         56648
cnt_days_cred_h_oper_3m    56648
start_cluster                  0
Length: 92, dtype: int64

In [22]:
df_test_float = test_df.select_dtypes(exclude='object').reset_index()  # Создаем датафрейм со столбцами численного типа И проверяем количесво пропусков
df_test_float = df_test_float.fillna(df_test_float.mean())                                     # Для последующего соединения таблиц установим столбец индексов
df_test_float.isna().sum()

index                      0
id                         0
balance_amt_avg            0
balance_amt_max            0
balance_amt_min            0
                          ..
cnt_deb_h_oper_3m          0
cnt_days_deb_h_oper_3m     0
sum_cred_h_oper_3m         0
cnt_cred_h_oper_3m         0
cnt_days_cred_h_oper_3m    0
Length: 83, dtype: int64

In [23]:
df_test_object = test_df.select_dtypes('object').reset_index() # Повторим запонление со столбцами типа object, используя моду
df_test_object = df_test_object.fillna(df_test_object.mode().iloc[0]).reset_index()
df_test_object.isna().sum()

level_0            0
index              0
date               0
channel_code       0
city               0
city_type          0
index_city_code    0
ogrn_month         0
ogrn_year          0
okved              0
segment            0
start_cluster      0
dtype: int64

In [24]:
# Объединяем численные и категориальные данные тестового датасета
test_df_modifide = df_test_float.merge(df_test_object, on=['index']).drop('index', axis=1)
test_df_modifide.head()

,id,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,ogrn_days_end_month,ogrn_days_end_quarter,ft_registration_date,max_founderpres,min_founderpres,...,date,channel_code,city,city_type,index_city_code,ogrn_month,ogrn_year,okved,segment,start_cluster
0,200000,-0.096224,0.335496,-0.125995,-0.095578,-1.533705,-1.683741,-0.653810,0.040347,0.025407,...,month_4,channel_code_12,city_14,city_type_0,index_city_code_46,ogrn_month_8,ogrn_year_11,okved_0,segment_3,{α}
1,200000,-0.024255,-0.059806,-0.124295,-0.023381,-1.533705,-1.683741,-0.636647,0.040347,0.025407,...,month_5,channel_code_12,city_14,city_type_0,index_city_code_46,ogrn_month_8,ogrn_year_11,okved_0,segment_3,{α}
2,200000,0.045988,0.049418,-0.125995,0.047079,-1.533705,-1.683741,-0.618911,0.040347,0.025407,...,month_6,channel_code_12,city_14,city_type_0,index_city_code_46,ogrn_month_8,ogrn_year_11,okved_0,segment_3,{α}
3,200001,-0.156722,-0.204920,-0.125856,-0.156258,0.092087,1.220030,-0.662964,0.040347,0.025407,...,month_4,channel_code_9,city_76,city_type_0,index_city_code_46,ogrn_month_10,ogrn_year_11,okved_6,segment_3,{α}
4,200001,-0.156722,-0.204920,-0.125856,-0.156258,0.092087,1.220030,-0.645800,0.040347,0.025407,...,month_5,channel_code_9,city_76,city_type_0,index_city_code_46,ogrn_month_10,ogrn_year_11,okved_6,segment_3,{α}


In [25]:
test_df_modifide.columns

Index(['id', 'balance_amt_avg', 'balance_amt_max', 'balance_amt_min',
       'balance_amt_day_avg', 'ogrn_days_end_month', 'ogrn_days_end_quarter',
       'ft_registration_date', 'max_founderpres', 'min_founderpres',
       'ogrn_exist_months', 'sum_of_paym_2m', 'sum_of_paym_6m',
       'sum_of_paym_1y', 'sum_a_oper_1m', 'cnt_a_oper_1m', 'sum_b_oper_1m',
       'cnt_b_oper_1m', 'sum_c_oper_1m', 'cnt_c_oper_1m', 'sum_deb_d_oper_1m',
       'cnt_deb_d_oper_1m', 'sum_cred_d_oper_1m', 'cnt_cred_d_oper_1m',
       'sum_deb_e_oper_1m', 'cnt_deb_e_oper_1m', 'cnt_days_deb_e_oper_1m',
       'sum_cred_e_oper_1m', 'cnt_cred_e_oper_1m', 'cnt_days_cred_e_oper_1m',
       'sum_deb_f_oper_1m', 'cnt_deb_f_oper_1m', 'cnt_days_deb_f_oper_1m',
       'sum_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'cnt_days_cred_f_oper_1m',
       'sum_deb_g_oper_1m', 'cnt_deb_g_oper_1m', 'cnt_days_deb_g_oper_1m',
       'sum_cred_g_oper_1m', 'cnt_cred_g_oper_1m', 'cnt_days_cred_g_oper_1m',
       'sum_deb_h_oper_1m', 'cnt_

In [26]:
# Удаляем столбцы из тестового датасета
test_df_modifide = test_df_modifide.drop(columns = ["channel_code", "city", 'index_city_code', 'ogrn_days_end_month','ogrn_days_end_quarter', 'ogrn_month', 'ft_registration_date', 'min_founderpres'])

In [27]:
test_df_modifide.shape

(290120, 85)

In [28]:
# Заменяем тип столбцов object на тип category
category_list = [
    "date", "city_type",
    "okved", "segment",
    "start_cluster", "ogrn_year",
]

test_df_modifide[category_list] = test_df_modifide[category_list].astype("category")

In [29]:
# Разбиваем данные на обучающую и валидационную выборки
X1 = test_df_modifide.drop(['id', 'start_cluster'], axis=1)
y1 = test_df_modifide['start_cluster']
x_train1, x_val1, y_train1, y_val1 = train_test_split(X1, y1,
                                                  test_size=0.2,
                                                  random_state=42)

In [30]:
# Обучаем модель
model.fit(x_train1, y_train1)

LGBMClassifier(n_jobs=-1, random_state=42, verbosity=-1)

In [31]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average='weighted')
    return sum(weights * classes_roc_auc)

In [32]:
cluster_weights = pd.read_excel('cluster_weights.xlsx').set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [33]:
# Вычисляем взвешенный ROC AUC для тестовых данных
y_pred_proba = model.predict_proba(x_val1)
weighted_roc_auc(y_val1, y_pred_proba, model.classes_, weights_dict)

0.8850785646379054